# Unit00｜Google Colab 環境設定與基本操作\n\n本 Notebook 的目標是：建立後續所有單元共用的執行環境（Drive 路徑、資料夾結構、套件版本、環境指紋），並養成一致的 Notebook 操作習慣。\n

## ✅ 本次完成清單\n\n- [ ] 確認 Python / Runtime 資訊（含 GPU）\n- [ ] 掛載 Google Drive（可保存資料、圖表、模型）\n- [ ] 建立課程資料夾結構（`data/`, `outputs/`, `models/`）\n- [ ] 安裝/檢查課程共用套件（NumPy/Pandas/Sklearn…）\n- [ ] 輸出環境指紋（方便 Debug 與作業驗收）\n

In [ ]:
# ===== 1) Runtime / Python / GPU 檢查 =====
import os
import platform
import sys
import textwrap
from datetime import datetime

print("Timestamp:", datetime.now().isoformat(timespec="seconds"))
print("Python:", sys.version.replace("\n", " "))
print("Platform:", platform.platform())
print("Working dir:", os.getcwd())

print("\nGPU check (nvidia-smi):")
exit_code = os.system("nvidia-smi > /dev/null 2>&1")
if exit_code == 0:
    os.system("nvidia-smi | head -n 20")
else:
    print("No GPU detected (this is OK for Part 0). If you need GPU: Runtime → Change runtime type → T4 GPU")


In [ ]:
# ===== 2) 掛載 Google Drive（只在 Colab 需要） =====
IN_COLAB = "google.colab" in sys.modules
print("IN_COLAB =", IN_COLAB)

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
else:
    print("Not running in Google Colab → skip drive.mount().")


In [ ]:
# ===== 3) 設定課程根目錄與資料夾結構 =====
# 建議把整門課的所有資料都放在同一個資料夾，後續 Unit 才不會找不到檔案

# 你可以改成自己的路徑，例如："/content/drive/MyDrive/FCU_CHE_AI"
COURSE_ROOT = "/content/drive/MyDrive/CHE-AI" if IN_COLAB else os.path.abspath("./CHE-AI")

DATA_DIR = os.path.join(COURSE_ROOT, "data")
OUTPUT_DIR = os.path.join(COURSE_ROOT, "outputs")
MODEL_DIR = os.path.join(COURSE_ROOT, "models")

for path in [COURSE_ROOT, DATA_DIR, OUTPUT_DIR, MODEL_DIR]:
    os.makedirs(path, exist_ok=True)

print("COURSE_ROOT:", COURSE_ROOT)
print("DATA_DIR:", DATA_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("MODEL_DIR:", MODEL_DIR)


In [ ]:
# ===== 4) 安裝/檢查課程共用套件 =====
# Colab 通常已內建常見套件，但版本可能不同；這裡用「最小共用集合」統一。

import subprocess

BASE_PACKAGES = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "scipy",
    "scikit-learn",
    "joblib",
    "tqdm",
    "openpyxl",
]

def pip_install(packages):
    cmd = [sys.executable, "-m", "pip", "install", "-q", "--upgrade"] + list(packages)
    print("Running:", " ".join(cmd[:8]), "... (")
    subprocess.check_call(cmd)

pip_install(BASE_PACKAGES)
print("Done.")


In [ ]:
# ===== 5) 版本檢查（環境指紋的一部分） =====
import importlib

PKG_TO_IMPORT = {
    "numpy": "numpy",
    "pandas": "pandas",
    "matplotlib": "matplotlib",
    "seaborn": "seaborn",
    "scipy": "scipy",
    "sklearn": "scikit-learn",
    "joblib": "joblib",
}

def get_version(module_name):
    mod = importlib.import_module(module_name)
    return getattr(mod, "__version__", "(no __version__)")

versions = {k: get_version(k) for k in PKG_TO_IMPORT.keys()}
for k, v in versions.items():
    print(f"{k:<10} {v}")


In [ ]:
# ===== 6)（可選）繪圖中文字顯示設定 =====
# Colab 若遇到中文亂碼，可嘗試安裝 Noto CJK 字型並設定 matplotlib。
# 若你不需要中文圖表，這格可以跳過。

import matplotlib.pyplot as plt

if IN_COLAB:
    # 安裝字型（只需做一次；若你覺得慢可以先略過）
    _ = os.system("apt-get -qq update && apt-get -qq install -y fonts-noto-cjk > /dev/null")
    plt.rcParams["font.family"] = ["Noto Sans CJK TC", "Noto Sans CJK SC", "DejaVu Sans"]
else:
    # 本機 Windows 常用字型
    plt.rcParams["font.sans-serif"] = ["Microsoft JhengHei", "Arial Unicode MS", "DejaVu Sans"]

plt.rcParams["axes.unicode_minus"] = False

plt.figure(figsize=(6, 3))
plt.plot([1, 2, 3], [1, 4, 2])
plt.title("中文測試：溫度 vs 時間")
plt.xlabel("時間 (min)")
plt.ylabel("溫度 (°C)")
plt.tight_layout()
plt.show()


In [ ]:
# ===== 7) 固定亂數種子（讓實驗更可重現） =====
import random
import numpy as np

os.environ["PYTHONHASHSEED"] = "0"
random.seed(0)
np.random.seed(0)

print("Seeds set.")


In [ ]:
# ===== 8) I/O 小測試：把輸出寫進 outputs/ =====
import pandas as pd

df = pd.DataFrame(
    {
        "time_min": [0, 1, 2, 3],
        "temperature_C": [25.0, 25.4, 26.1, 26.0],
        "pressure_bar": [1.0, 1.1, 1.2, 1.15],
    }
)

out_csv = os.path.join(OUTPUT_DIR, "unit00_io_test.csv")
df.to_csv(out_csv, index=False)

print("Saved:", out_csv)
print(pd.read_csv(out_csv).head())


## ✅ 你現在可以開始做後續單元了\n\n之後所有 Unit 的 Notebook，建議直接複製以下三段「固定開頭」：\n\n1) Drive mount（只在 Colab）\n\n2) 設定 `COURSE_ROOT / DATA_DIR / OUTPUT_DIR / MODEL_DIR`\n\n3) 需要時才做 `pip install`（避免每次都花很多時間）\n